In [ ]:
library(tidyverse)
library(DESeq2)

In [ ]:
# TODO: Consider filtering counts (e.g. To remove rows with fewer than 10 reads)
cts <- read_delim(snakemake@input[["counts"]],delim = "\t", skip = 1) %>% 
    rename_with(., ~ stringr::str_extract(pattern = "(?<=\\/)\\w{4}-\\d{1}-\\w+-Seq(?=\\/)", string = .x), contains("out.bam")) %>% 
    column_to_rownames("Geneid") %>% 
    mutate(total_reads = rowSums(select(., contains("Seq")))) %>% 
    # filter(total_reads >= 10) %>% 
    dplyr::select(-Chr, -Start, -End, -Strand, -Length, -total_reads) %>% 
    as.matrix(.)

In [ ]:
head(cts)

In [ ]:
samples <- read_delim(snakemake@config[["sample_sheet"]], delim = "\t")
samples

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = samples,
                              design = ~ cyanotype + tissue)
dds$group <- factor(paste0(dds$cyanotype, dds$tissue))
design(dds) <- ~ group
dds

In [ ]:
dds <- DESeq(dds)

In [ ]:
resultsNames(dds)

In [ ]:
ac_gene <- "ACLI19_g12956"
li_gene <- "ACLI19_g37877"

In [ ]:
res <- results(dds, contrast = c("group", "AcLiFlower", "AcLiRoot"))
res[li_gene,]

In [ ]:
# TODO: Try different shrinkage estimators
# Note 'apeglm' is the default
# resLFC <- lfcShrink(dds, coef=resultsNames(dds)[2], type="apeglm")
# resLFC

In [ ]:
summary(res)

In [ ]:
plotMA(res)

In [ ]:
d <- plotCounts(dds, gene=li_gene, intgroup=c("tissue", "cyanotype"), returnData=TRUE)

ggplot(d, aes(x = tissue, y = count)) + 
    geom_point(aes(shape = cyanotype, color = cyanotype, fill = cyanotype), size = 4.5) +
    ylab("Normalized read count") + xlab("Tissue") +
    theme_classic() +
    theme(axis.text = element_text(size = 14),
          axis.title = element_text(size = 17),
          legend.text = element_text(size = 12),
          legend.title = element_text(size = 14))

In [ ]:
d